<a href="https://colab.research.google.com/github/sagihaider/Biogas_Prediction/blob/master/main_Biogas_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/Biogas_Prediction.git

Cloning into 'Biogas_Prediction'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [2]:
import numpy as np
import pandas as pd

df=pd.read_excel('/content/Biogas_Prediction/Data/Gasification Data_Daya.xlsx', index_col=None, header=None)
df1 = df.iloc[1:223,1:15] 
print(df1)


print(df1.isnull().values.any())
print(df1.isna().values.any())

# Drop null balues and store dataframe in dataframe 2
df2=df1.dropna()
print(df2)

#Check Null values again after removing
print(df2.isnull().values.any())
print(df2.isna().values.any())

# Store Dependent and independent variabels in X and y
X= df2.iloc[0:187,0:9]
y = df2.iloc[0:187,9:14]

# Convert X and Y in Numpy Array
X=X.to_numpy()
y=y.to_numpy()


        1     2      3     4   ...       11       12       13        14
1       50   5.7   44.1  23.5  ...  36.1757  34.8837  7.75194       2.3
2       50   5.7   44.1    21  ...  31.1751  35.9712  6.47482       2.5
3       50   5.7   44.1    23  ...  29.8507  35.8209  7.16418       2.5
4       50   5.7   44.1    22  ...  31.1751  35.9712  6.47482       2.1
5       50   5.7   44.1    25  ...  31.1751  35.9712  6.47482       2.4
..     ...   ...    ...   ...  ...      ...      ...      ...       ...
218  46.85   6.3  45.55     9  ...  35.1087  30.5865   10.465  0.858789
219  46.85   6.3  45.55     9  ...  33.3062  32.8614  9.70615  0.947532
220  46.85   6.3  45.55     9  ...  33.7895  35.4904  9.45507  0.975752
221  46.76  5.68  39.46  9.94  ...  31.9511  32.7829  9.93368  0.971784
222  46.76  5.68  39.46  9.94  ...  29.1236  34.4266   8.7784  0.982989

[222 rows x 14 columns]
True
True
        1     2      3     4   ...       11       12       13        14
1       50   5.7   44.1  23.5

In [3]:
from numpy import asarray
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')

n_features=1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
n_inputs, n_outputs = X.shape[1], y.shape[1]

In [5]:
# get the model
def get_model(n_inputs, n_outputs):
  # model = Sequential()
  # model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_inputs, n_features)))
  # model.add(Conv1D(filters=8, kernel_size=2, activation='relu'))
  # model.add(Dropout(0.5))
  # model.add(MaxPooling1D(pool_size=2))
  # model.add(Flatten())
  # model.add(Dense(100, activation='relu'))
  # model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
  # model.compile(loss='mae', optimizer='adam')
  # model.summary()
  # return model

  # define model
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_inputs, n_features)))
  model.add(MaxPooling1D(pool_size=2))
  # model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(n_outputs))
  model.compile(optimizer='adam', loss='mse')
  model.summary()
  return model

In [6]:
model = get_model(n_inputs, n_outputs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 64)             192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                12850     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 13,297
Trainable params: 13,297
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X_train, y_train, 
                    epochs=1000, 
                    batch_size=32, 
                    validation_data=(X_test, y_test), verbose=0)

In [8]:
yhat = model.predict(X_test)
# mae = model.evaluate(X_test, y_test, verbose=0)

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, yhat, multioutput='raw_values')

array([36.237057  , 27.660988  , 32.199512  , 11.443638  ,  0.79450524],
      dtype=float32)